In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from __future__ import print_function

dataset = pd.read_csv("C:/Users/renan/Desktop/UFRGS/Machine Learning/Trabalho1/random-forests-1/datasets/benchmark.csv") 

dir(dataset)

print ('Banco com %d amostras e %d colunas\n' % dataset.shape)

print ('Tabela sumarizando as colunas do banco\n')
dataset.describe()

Banco com 14 amostras e 5 colunas

Tabela sumarizando as colunas do banco



,Tempo,Temperatura,Umidade,Ventoso,Joga
count,14,14,14,14,14
unique,3,3,2,2,2
top,Ensolarado,Amena,Alta,Falso,Sim
freq,5,6,7,8,9


In [ ]:
def bootstrap_sample(dataset):
    bootstrap = np.random.choice(dataset, size=len(dataset), replace=True) 
    return bootstrap

for data in dataset:
    print(bootstrap_sample(data))

#teste para o codigo com dados tuple

In [3]:
def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count for subset in subsets )

def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values()) # retorna a media dos sub-conjuntos resultantes

def gain(class_entropy, mean_entropy):
    return class_entropy - mean_entropy;

def class_entropy(inputs):
    """retorna a entropia sobre a classe"""
    class_values = []
    for item in inputs:
        class_values.append(item[1]) # suponto a definição atual dos dados!
    probabilities = class_probabilities(class_values)
    class_entropy = entropy(probabilities)
    return class_entropy

def build_tree_id3(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues

    if num_trues == 0:                  # if only Falses are left
        return False                    # return a "False" leaf

    if num_falses == 0:                 # if only Trues are left
        return True                     # return a "True" leaf

    if not split_candidates:            # if no split candidates left
        return num_trues >= num_falses  # return the majority leaf

    # otherwise, split on the best attribute
    
    best_gain = 0
    best_attribute = None
    for candidate in split_candidates:
        mean_entropy = partition_entropy_by(inputs, candidate)
        print('[INFO] Atributo: ', candidate, '- entropia média: ', mean_entropy)
        gain_result = gain(class_entropy(inputs), mean_entropy)
        print('[INFO] Atributo: ', candidate, '- ganho: ', gain_result)
        if(gain_result > best_gain):
            best_gain = gain(class_entropy(inputs), mean_entropy)
            best_attribute = candidate
        print('--')
    
    print('----------------------------------------')
    print('[INFO] best_attribute: ', best_attribute)
    print('----------------------------------------')
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    #subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

if __name__ == "__main__":

    # inputs contendo tupulas contendo dicionarios
    inputs = [
        ({'Tempo':'Ensolarado','Temperatura':'Quente','Umidade':'Alta','Ventoso':'no'}, False),
        ({'Tempo':'Ensolarado','Temperatura':'Quente','Umidade':'Alta','Ventoso':'yes'}, False),
        ({'Tempo':'Nublado','Temperatura':'Quente','Umidade':'Alta','Ventoso':'no'}, True),
        ({'Tempo':'Chuvoso','Temperatura':'Amena','Umidade':'Alta','Ventoso':'no'}, True),
        ({'Tempo':'Chuvoso','Temperatura':'Fria','Umidade':'Normal','Ventoso':'no'}, True),
        ({'Tempo':'Chuvoso','Temperatura':'Fria','Umidade':'Normal','Ventoso':'yes'}, False),
        ({'Tempo':'Nublado','Temperatura':'Fria','Umidade':'Normal','Ventoso':'yes'}, True),
        ({'Tempo':'Ensolarado','Temperatura':'Amena','Umidade':'Alta','Ventoso':'no'}, False),
        ({'Tempo':'Ensolarado','Temperatura':'Fria','Umidade':'Normal','Ventoso':'no'}, True),
        ({'Tempo':'Chuvoso','Temperatura':'Amena','Umidade':'Normal','Ventoso':'no'}, True),
        ({'Tempo':'Ensolarado','Temperatura':'Amena','Umidade':'Normal','Ventoso':'yes'}, True),
        ({'Tempo':'Nublado','Temperatura':'Amena','Umidade':'Alta','Ventoso':'yes'}, True),
        ({'Tempo':'Nublado','Temperatura':'Quente','Umidade':'Normal','Ventoso':'no'}, True),
        ({'Tempo':'Chuvoso','Temperatura':'Amena','Umidade':'Alta','Ventoso':'yes'}, False)
    ]
    
    tree = build_tree_id3(inputs)
    print('\n')
    print('----------------------------------------')
    print("[INFO] Árvore construida:")
    print('----------------------------------------')
    print(tree)

NameError: name 'defaultdict' is not defined

In [14]:
def entropia(dataset):

    """
    Calculo da entropia do conjunto de dados.
    Recebe como parâmetro o dataset. 
    """

    classe = dataset.keys()[-1]   #Define a variável alvo como classe
    entropy = 0
    values = dataset[classe].unique()
    for value in values:
        fraction = dataset[classe].value_counts()[value]/len(dataset[classe])
        entropy += -fraction*np.log2(fraction)
    return entropy

#Teste dos valores gerados pela função 
teste_entropia = entropia(dataset)
print (teste_entropia)

0.9402859586706311


In [20]:
eps = np.finfo(float).eps
from numpy import log2 as log

def entropia_atributos(dataset, attribute):
    
    classe = dataset.keys()[-1]   
    target_variables = dataset[classe].unique()  
    variables = dataset[attribute].unique()  
    entropy_attribute = 0
    
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(dataset[attribute][dataset[attribute] == variable][dataset[classe] == target_variable]) 
            den = len(dataset[attribute][dataset[attribute] == variable]) 
            fraction = num/(den + eps)  
            entropy_each_feature += -fraction*log(fraction + eps) 
        fraction2 = den/len(dataset)
        entropy_attribute += -fraction2*entropy_each_feature

    return(abs(entropy_attribute))

#Teste dos valores gerados pela função 
teste_entrobia_atributos = {k:entropia_atributos(dataset,k) for k in dataset.keys()[:-1]}
print (teste_entrobia_atributos)

{'Tempo': 0.6935361388961914, 'Temperatura': 0.9110633930116756, 'Umidade': 0.7884504573082889, 'Ventoso': 0.892158928262361}


In [1]:
def ganho_de_informacao(dataset):

    IG = []

    for key in dataset.keys()[:-1]:
        IG.append(entropia(dataset)-entropia_atributos(dataset,key))
    return dataset.keys()[:-1][np.argmax(IG)]

#Teste dos valores gerados pela função
IG = {k:entropia_atributos(teste_entropia,teste_entrobia_atributos[k]) for k in teste_entrobia_atributos}
print (dataset.keys()[:-1][np.argmax(IG)])

NameError: name 'teste_entrobia_atributos' is not defined

In [ ]:
def gerar_particao(dataset, node, value):
  return dataset[dataset[node] == value].reset_index(drop=True)
  
  
def decision_tree(dataset, tree = None): 
    
    classe = dataset.keys()[-1]
    node = ganho_de_informacao(dataset)
    attValue = np.unique(dataset[node])
        
    if tree is None:                    
        tree={}
        tree[node] = {} 

    for value in attValue:
        
        subtable = gerar_particao(dataset, node, value)
        clValue,counts = np.unique(subtable['Eat'],return_counts=True)                        
        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = decision_tree(subtable)
                   
    return tree

teste = decision_tree(dataset)